In general, if df is a Pandas object (such as a dataframe or series), then the Python command df.x(y, w=z) becomes x(df, y, w=z) in Julia. df.loc[a,b,c] becomes loc(df)[a,b,c] (same for iloc and ix). 

In [2]:
import Pkg
using Pkg
#Pkg.add("")
using Pandas,FreqTables, Distributions, BlackBoxOptim, ForwardDiff, Optim, HTTP

In [15]:
dataset = read_csv("datamerged2.csv");

In [16]:
subset1 = query(dataset,("treatment==1 and (round==2 or round==3)"));

In [10]:
describe(subset1)

,id,treatment,tofrom,value,role,resp,round,rounds,old
count,422.000000,422.0,422.000000,422.000000,422.0,422.000000,422.000000,422.000000,422.000000
mean,124.805687,1.0,135.838863,20.284550,0.0,0.819905,2.177725,2.355450,0.867299
std,81.297525,0.0,85.181224,6.608621,0.0,0.384722,0.382735,0.479218,0.676319
min,1.000000,1.0,6.000000,4.200000,0.0,0.000000,2.000000,2.000000,0.000000
25%,25.000000,1.0,28.000000,15.000000,0.0,1.000000,2.000000,2.000000,0.000000
50%,152.000000,1.0,167.000000,20.000000,0.0,1.000000,2.000000,2.000000,1.000000
75%,162.000000,1.0,177.000000,25.000000,0.0,1.000000,2.000000,3.000000,1.000000
max,250.000000,1.0,274.000000,45.000000,0.0,1.000000,3.000000,3.000000,2.000000


In [33]:
# Offers in vector form
s = values(subset1["value"]);

In [19]:
#parameters
alph = 1 #competitiveness term
r = 5; #reference offer (close to the minimum offer observed in the data)

In [31]:
# Distribution of our loss aversion paramters
# Initial guess is a Truncated Normal Distribution with(mean=3, std=1, lb=1, ub=inf)
F = TruncatedNormal(3,1,1,Inf);

In [21]:
#probability loss aversion param below 2.5 given mean and std
cdf(F,2.5)

0.29244046596551004

In [20]:
#probability loss aversion param is equal to 2.5 given mean and std
pdf(F,2.5)

0.3602613193145302

### Stuff To Discuss

In [29]:
#summary stats of the distribution of offers
mu = mean(s)
sigma = std(s);

In [24]:
#guess the distribution of offers as a truncated normal
Q = TruncatedNormal(mu, sigma, 0, 55);

Truncated(Normal{Float64}(μ=20.284549780931517, σ=6.608621288880517); lower=0.0, upper=55.0)

In [25]:
#generate the probability of seeing a given offer 's' (PDF)
pdfValues = zeros(0)
for i in s
    append!(pdfValues, pdf(Q, i))
end

In [35]:
#generate the probability of seeing an offer *less* than a given offer 's' (CDF)
cdfValues = zeros(0)
for i in s
    append!(cdfValues, cdf(Q, i))
end

In [30]:
#loss aversion parameters
lambda = 2*cdfValues*(1 + 2*alph)./pdfValues - 2*(1 - s*(1 + 2*alph)) + 1
mean(lambda)

LoadError: MethodError: no method matching -(::Int64, ::Vector{Float64})
For element-wise subtraction, use broadcasting with dot syntax: scalar .- array
[0mClosest candidates are:
[0m  -(::Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8}) at C:\Users\Alex\AppData\Local\Programs\Julia-1.7.1\share\julia\base\int.jl:85
[0m  -(::T, [91m::T[39m) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} at C:\Users\Alex\AppData\Local\Programs\Julia-1.7.1\share\julia\base\int.jl:86
[0m  -([91m::SparseArrays.AbstractSparseMatrixCSC[39m, ::Array) at C:\Users\Alex\AppData\Local\Programs\Julia-1.7.1\share\julia\stdlib\v1.7\SparseArrays\src\sparsematrix.jl:1776
[0m  ...

$$
\lambda = \frac{2Q(s)(1+2\alpha)}{Q^\prime(s)} - 2(1-s(1+2\alpha)) + 1 
$$